# INDR344 HW04 - Dump Truck Model

# Özge Sayar

In [1]:
import numpy as np
import simpy as sp

In [2]:
#generating loading time
def load_time():
    loading = np.random.rand()
    if loading < 0.30:
        return(5)
    if loading < 0.80:
        return(10)
    if loading < 1:
        return(15)

#generating Weighing time
def weighing_time():
    weighing = np.random.rand()
    if weighing < 0.70:
        return(12)
    if weighing < 1:
        return(16)
    
#generating travel time
def travel_time():
    travel = np.random.rand()
    if travel < 0.40:
        return(40)
    if travel < 0.70:
        return(60)
    if travel < 0.90:
        return(80)
    if travel < 1:
        return(100)
    

In [12]:
number_of_loadings_started = 0
total_queue_time_for_loading = []
number_of_loadings_completed = 0
total_loading_time = []
number_of_weighings_started = 0
total_queue_time_for_weighing = []
number_of_weighings_completed = 0
total_weighing_time = []
traveling_exit_time = 0

def truck(env, name, loaders, scaler):
    global number_of_loadings_started 
    global total_queue_time_for_loading 
    global number_of_loadings_completed 
    global total_loading_time 
    global number_of_weighings_started 
    global total_queue_time_for_weighing
    global number_of_weighings_completed 
    global total_weighing_time 
    
    while True: #dump trucks coming with a continuous infinite loop
        
        loading_queue_enter_time = env.now
        print("{:s} arrives at the loader at {:.2f}.".format(name, env.now))
        loader_request = loaders.request()
        yield loader_request

        loading_enter_time = env.now
        print("{:s} enters the loader at {:.2f}.".format(name, env.now))
        number_of_loadings_started = number_of_loadings_started + 1
        loading_time = load_time()
        total_queue_time_for_loading.append(loading_enter_time - loading_queue_enter_time)
        yield env.timeout(loading_time)

        loading_exit_time = env.now
        print("{:s} leaves the loader at {:.2f}.".format(name, env.now))
        number_of_loadings_completed = number_of_loadings_completed + 1 
        loaders.release(loader_request)

        total_loading_time.append(loading_exit_time - loading_enter_time)
        
        scaling_queue_enter_time = env.now
        print("{:s} arrives at the scale at {:.2f}.".format(name, env.now))
        scaler_request = scaler.request()
        yield scaler_request

        scaling_enter_time = env.now
        print("{:s} enters the scaler at {:.2f}.".format(name, env.now))
        number_of_weighings_started = number_of_weighings_started + 1
        scaling_time = weighing_time()
        total_queue_time_for_weighing.append(scaling_enter_time -scaling_queue_enter_time )
        yield env.timeout(scaling_time)

        number_of_weighings_completed = number_of_weighings_completed +1 
        print("{:s} leaves the scaler at {:.2f}.".format(name, env.now))
        scaling_exit_time = env.now
        scaler.release(scaler_request)

        total_weighing_time.append(scaling_exit_time - scaling_enter_time)
    
        print("{:s} starts traveling at {:.2f}.".format(name, env.now))
        traveling_time = travel_time()
        yield env.timeout(traveling_time)
        traveling_exit_time = env.now


        
def setup(env):
#first come first serve
    loaders = sp.Resource(env, 2)
    scaler = sp.Resource(env, 1)

    for i in np.arange(1, 7): #there are 6 trucks at the loaders at time 0
        env.process(truck(env, "Truck#{:d}".format(i), loaders, scaler))
        yield env.timeout(traveling_exit_time)
        
        
np.random.seed(344)
env = sp.Environment()
env.process(setup(env))
env.run(until = 480)

print("--------------------------------------------------------------------------------------------------------------")
print("Number of loadings started", number_of_loadings_started)
print("Average queue time for loading is", np.around(np.mean(total_queue_time_for_loading), decimals=2))
print("Number of loadings completed", number_of_loadings_completed)
print("Average loading time is", np.around(np.mean(total_loading_time),decimals=2))
print("Number of weighings started", number_of_weighings_started)
print("Average queue time for weighing is", np.mean(total_queue_time_for_weighing))
print("Number of weighings completed", number_of_weighings_completed)
print("Average weighing time is", np.around(np.mean(total_weighing_time), decimals=2))

Truck#1 arrives at the loader at 0.00.
Truck#2 arrives at the loader at 0.00.
Truck#1 enters the loader at 0.00.
Truck#3 arrives at the loader at 0.00.
Truck#2 enters the loader at 0.00.
Truck#4 arrives at the loader at 0.00.
Truck#5 arrives at the loader at 0.00.
Truck#6 arrives at the loader at 0.00.
Truck#1 leaves the loader at 10.00.
Truck#1 arrives at the scale at 10.00.
Truck#2 leaves the loader at 10.00.
Truck#2 arrives at the scale at 10.00.
Truck#1 enters the scaler at 10.00.
Truck#3 enters the loader at 10.00.
Truck#4 enters the loader at 10.00.
Truck#3 leaves the loader at 15.00.
Truck#3 arrives at the scale at 15.00.
Truck#4 leaves the loader at 15.00.
Truck#4 arrives at the scale at 15.00.
Truck#5 enters the loader at 15.00.
Truck#6 enters the loader at 15.00.
Truck#1 leaves the scaler at 22.00.
Truck#1 starts traveling at 22.00.
Truck#2 enters the scaler at 22.00.
Truck#5 leaves the loader at 25.00.
Truck#5 arrives at the scale at 25.00.
Truck#6 leaves the loader at 25.00